# 셀레니움으로 자료 가져오기
* chrome 웹드라이버설치 [설치사이트]( https://chromedriver.chromium.org/downloads)
* 셀레니움참고 [참고사이트](https://galid1.tistory.com/478)

In [25]:
# #선행설치
# !pip install selenium


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

import time
import pdftotext # pdf 읽는용도
import pikepdf #lock걸려있는 pdf해제
import fitz # PyMuPDF

import pandas as pd
import numpy as np

import os
import re

# 본문내용긁어오기

In [84]:
#1회 사이클 이후 느려지는 경향이 있어 1회 약 10호 반복후 저장하는 형태로 변경 -> 다시 초기화(여기서하면 1ho씩 저장됨)

#저장위치
final_dict = dict()
final_dict['ho'] = []
final_dict['dates'] = []
final_dict['category'] = []
final_dict['city'] = []
final_dict['title'] = []
final_dict['contents'] = []

def get_contents():    

    try :
        for j in range(2,30):
            one_title = driver.find_element_by_xpath('//*[@id="container_suite"]/div[4]/div/div/ul/li[{num}]/div[1]/h2/a'.format(num =j))
            one_title.click()

            article_xpath = driver.find_element_by_xpath('//*[@id="center"]/article')

            #facebook, Twitter의 위치가 계속 변동되어 지우고 위치지정
            article = article_xpath.text.split('\n')
            article.remove('Facebook')
            article.remove('Twitter')

            #추출
            title = article[0]
            dates = article[2]
            ho = article[6]
            category = article[8]
            contents = article[9:-1]

            city = title.split(" ")[-1]
            if ')' in city:
                city = city.replace('(',"").replace(')',"")
            else :
                city = np.nan

            #저장 
            final_dict.get('dates',0).append(dates)
            final_dict.get('ho',0).append(ho)
            final_dict.get('category',0).append(category)
            final_dict.get('city',0).append(city)
            final_dict.get('title',0).append(title)
            final_dict.get('contents',0).append(contents)

            #뒤로가기
            time.sleep(0.5)
            driver.back()
    
    except Exception as e:
        pass 


In [85]:
def save_reset():
    
    global final_dict
    #csv 저장
    final_df = pd.DataFrame(final_dict)

    start_ho = final_df.ho.iloc[0]
    end_ho = final_df.ho.iloc[-1]

    final_df.to_csv('All_document({s}-{e}).csv'.format(s = start_ho, e = end_ho))
    
    #변수초기화
    final_dict = dict()
    final_dict['ho'] = []
    final_dict['dates'] = []
    final_dict['category'] = []
    final_dict['city'] = []
    final_dict['title'] = []
    final_dict['contents'] = []

# 과정통합 
* 오래걸림 따라서 분할작업진행

In [28]:
#전체 진행 (오류나면서 끝나는것이 맞다)
os.chdir('D:/Python/SDI 셀레니움/')
options = Options()
# chrome을 전체화면으로/ headless
options.add_argument('--start-maximized')
options.add_argument('--headless')
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#사이트이동
driver.get('http://www.sdi.re.kr')
time.sleep(2)

#세계도시동향 클릭
donghyang = driver.find_element_by_xpath('//*[@id="block-system-main-menu"]/div/ul/li[2]/a')
donghyang.click()
time.sleep(2)

#1페이지 내 다운로드 
for i in range(1,10):
    #처음페이지는 9개인데 이후 10개씩
    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
    ho.click()
    time.sleep(1)
    get_contents()
# 2페이지로 넘어가는게 아래의 숫자와 같고 9가 반복됨 (원인은 숫자가 계속 밀리기 때문)
#해당페이지는 2,3,4,5,6,7,
for page_num in [2,5,6,7,8,9]:
# for page_num in [2]:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[{page_num}]/a'.format(page_num = page_num))
    next_page.click()
    time.sleep(1)
    try :
        for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
            ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
            ho.click()
            time.sleep(1)
            get_contents()
    except:
        pass
#이후 반복 2020-05-07 11:42:15 while True 이중으로 썻던 이유가 기억안난다
while True:
    try:
        #이후반복
        while True:
            next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[9]/a')
            next_page.click()
            driver.implicitly_wait(10)
            for i in range(1,11):
                try:
            #처음페이지는 9개인데 이후 10개씩
                    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                    ho.click()
                    time.sleep(1)
                    get_contents()
                except:
                    pass
    except:
        #에러난 페이지 내 다운로드 
        ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[2]/ul/li[1]/a')
        ho.click()
        for i in range(1,11):
            try:
                #처음페이지는 9개인데 이후 10개씩
                ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                ho.click()
                time.sleep(1)
                get_contents()
            except: 
                pass
        pass


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: unhandled inspector error: {"code":-32000,"message":"Inspected target navigated or closed"}
  (Session info: chrome=81.0.4044.129)


### 특정페이지 이후(분할작업) 9~20

In [ ]:
#전체 진행 (오류나면서 끝나는것이 맞다)
os.chdir('D:/Python/SDI 셀레니움/')
options = Options()
# chrome을 전체화면으로/ headless
options.add_argument('--start-maximized')
options.add_argument('--headless')
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#사이트이동
for num in range(9, 21):
    driver.get('https://www.si.re.kr/world_trends/22383?paged={page_num}'.format(page_num = num))
    time.sleep(2)

    #1페이지 내 다운로드 
    for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
        ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
        ho.click()
        time.sleep(1)
        get_contents()
    save_reset()
    #이후반복
    while True:
        next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[9]/a')
        next_page.click()
        driver.implicitly_wait(10)
        for i in range(1,11):
            try:
        #처음페이지는 9개인데 이후 10개씩
                ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
                ho.click()
                time.sleep(1)
                get_contents()
            except:
                pass
        save_reset()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


### 디버깅 : 에러나는 페이지 5번(434포함됨) 확인
-> 1페이지 긁을때는 문제 없음

In [50]:
#저장위치
final_dict = dict()
final_dict['ho'] = []
final_dict['dates'] = []
final_dict['category'] = []
final_dict['city'] = []
final_dict['title'] = []
final_dict['contents'] = []

In [33]:
#1페이지 내 다운로드 
for i in range(1,10):
    #처음페이지는 9개인데 이후 10개씩
    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
    ho.click()
    time.sleep(0.5)
    get_contents()

In [38]:
pd.options.display.max_rows = 10

In [39]:
pd.DataFrame(final_dict)

,ho,dates,category,city,title,contents
0,제436호,2018.08.08,헤드라인뉴스,교토市,中企에 에너지 관리전문가 무료 파견 (일본 교토市),[일본 교토市는 중소기업에 에너지 관리전문가를 무료로 파견해 에너지절약계획의 준비부...
1,제436호,2018.08.08,사회/복지,파리市,공유경제 관련 세대간 유대 강화 등 2가지 제안 (프랑스 파리市),[프랑스 파리市 ‘미래세대회의’(CGF: Conseil des Generations...
2,제436호,2018.08.08,사회/복지,베를린市,‘새 이웃 개발하기’ 프로젝트 거점공간 4곳 추가 (독일 베를린市),[독일 베를린市는 최근 수년간 유입된 많은 난민이 베를린에 오래 머무를 것으로 예상...
3,제436호,2018.08.08,사회/복지,자카르타州,"지역보건소, ‘정신질환 대응’ 모바일 앱 개발 (인도네시아 자카르타州)",[인도네시아 자카르타州 남자카르타市 칠란닥(Cilandak)區 보건소는 정신건강 관...
4,제436호,2018.08.08,행재정/교육,런던市,공공병원 응급실 폭력행위 줄이는 디자인 프로그램 (영국 런던市),[영국 런던市는 공공병원 응급실에서 발생하는 폭력･폭언 등 공격성 사건이 응급실 내...
...,...,...,...,...,...,...
85,제428호,2018.04.12,사회/복지,버밍엄市,‘아동 성보호 인식 확산’ 캠페인 진행 (영국 버밍엄市),[전 세계적인 미투(#MeToo) 운동 확산으로 사회적 약자를 대상으로 한 성범죄에...
86,제428호,2018.04.12,사회/복지,빅토리아州,장애인 고교생에 유급 고용기회 제공 (호주 빅토리아州),[호주 빅토리아州는 장애인의 학창시절 고용경험이 성인기의 고용에 가장 큰 영향을 미...
87,제428호,2018.04.12,환경/안전,바르셀로나市,분리수거 방법 개선 등으로 ‘쓰레기 무배출’ 추진 (스페인 바르셀로나市),[스페인 바르셀로나市는 ‘쓰레기 무배출’(Residuo Cero: Zero Wast...
88,제428호,2018.04.12,환경/안전,오사카市,재해 발생 시 임산부 등 피난약자에 충분한 지원 (일본 오사카市),"[일본 오사카市는 재해 발생에 따른 피난 상황에서 도움이 필요한 고령자, 장애인, ..."


### 디버그2 - 페이지 이동문제
-> 이동자체는 문제없음

In [43]:
for page_num in [2,5,6,7,8,9]: #7번페이지까지 이동 
# for page_num in [2]:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[{page_num}]/a'.format(page_num = page_num))
    next_page.click()
    time.sleep(0.5)

### 디버그3 - 페이지 이동후 긁는것 문제(유력)
->문제의 5번페이지 긁는데 이상이 없다 : 원인을 모르겠다

In [44]:
for page_num in [2,5,6,7,8,9]:
# for page_num in [2]:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[{page_num}]/a'.format(page_num = page_num))
    next_page.click()
    time.sleep(0.5)
    try :
        for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
            ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
            ho.click()
            time.sleep(0.5)
            if page_num == (7 or 8):
                get_contents()
    except:
        pass

In [48]:
aa = pd.DataFrame(final_dict)

In [49]:
aa.drop_duplicates('title')

,ho,dates,category,city,title,contents
0,제436호,2018.08.08,헤드라인뉴스,교토市,中企에 에너지 관리전문가 무료 파견 (일본 교토市),[일본 교토市는 중소기업에 에너지 관리전문가를 무료로 파견해 에너지절약계획의 준비부...
1,제436호,2018.08.08,사회/복지,파리市,공유경제 관련 세대간 유대 강화 등 2가지 제안 (프랑스 파리市),[프랑스 파리市 ‘미래세대회의’(CGF: Conseil des Generations...
2,제436호,2018.08.08,사회/복지,베를린市,‘새 이웃 개발하기’ 프로젝트 거점공간 4곳 추가 (독일 베를린市),[독일 베를린市는 최근 수년간 유입된 많은 난민이 베를린에 오래 머무를 것으로 예상...
3,제436호,2018.08.08,사회/복지,자카르타州,"지역보건소, ‘정신질환 대응’ 모바일 앱 개발 (인도네시아 자카르타州)",[인도네시아 자카르타州 남자카르타市 칠란닥(Cilandak)區 보건소는 정신건강 관...
4,제436호,2018.08.08,행재정/교육,런던市,공공병원 응급실 폭력행위 줄이는 디자인 프로그램 (영국 런던市),[영국 런던市는 공공병원 응급실에서 발생하는 폭력･폭언 등 공격성 사건이 응급실 내...
...,...,...,...,...,...,...
182,제427호,2018.03.28,사회/복지,에든버러市,2011년부터 청년에 다양한 일자리·교육기회 제공 (영국 에든버러市),[영국 에든버러市는 세계적 금융위기로 경제상황이 악화되던 2011년부터 고등학교를 ...
183,제427호,2018.03.28,사회/복지,뉴욕市,시민 정신건강문제 해결에 정책적 대응 강화 (미국 뉴욕市),[성인 5명 중 1명 이상이 정신건강문제를 겪고 있는 미국 뉴욕市는 2016년부터 ...
184,제427호,2018.03.28,환경/안전,파리市,‘생물다양성 증진’ 2018~2024년 플랜 발표 (프랑스 파리市),[프랑스 파리市는 ‘일 드 프랑스’(ile de France: 수도권)를 포함한 ‘...
185,제427호,2018.03.28,도시계획/주택,바르셀로나市,건물에너지 효율적 사용 정보 다룬 안내서 발간 (스페인 바르셀로나市),[스페인 바르셀로나市는 기후변화에 대처하는 프로젝트의 하나로 건물에너지를 효율적으로...


### 분할작업

In [53]:
#1페이지 내 다운로드 
for i in range(1,10):
    #처음페이지는 9개인데 이후 10개씩
    ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
    ho.click()
    time.sleep(1)
    get_contents()
# 2페이지로 넘어가는게 아래의 숫자와 같고 9가 반복됨 (원인은 숫자가 계속 밀리기 때문)
#해당페이지는 2,3,4,5,6,7,
for page_num in [2,5,6,7,8,9]:
# for page_num in [2]:
    next_page = driver.find_element_by_xpath('//*[@id="container_suite"]/div[5]/ul/li[{page_num}]/a'.format(page_num = page_num))
    next_page.click()
    time.sleep(1)
    try :
        for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
            ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
            ho.click()
            time.sleep(1)
            get_contents()
    except:
        pass

In [55]:
aa = pd.DataFrame(final_dict)

In [57]:
aa.shape, aa.drop_duplicates('title').shape

((607, 6), (602, 6))

In [74]:
    #csv 저장
    final_df = pd.DataFrame(final_dict)

    start_ho = final_df.ho.iloc[0]
    end_ho = final_df.ho.iloc[-1]

    final_df.to_csv('All_document({s}-{e}).csv'.format(s = start_ho, e = end_ho))